In [ ]:
# Make a summary file of the sample response raw data

In [3]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from scipy.optimize import curve_fit
import numpy as np # need for linspace in plot
from statistics import mean
import statistics
import time
pd.options.display.max_rows = 500

In [51]:
path_list = sorted(glob.glob('../20210724_exacloud_results/*.tsv'))
cohort_list = []
for path in path_list:
    cohort = path.split('/')[2].split('.')[0]
    if cohort not in cohort_list:
        cohort_list.append(cohort)

In [82]:
start = time.time()
store_frame = pd.DataFrame(columns=['Cohort',
                                'Sample_counts',
                                'Average_scores',
                                'Resample_by_step',
                                'Stdev_step'])
for cohort in cohort_list:
    cohort_path_block = sorted(
        glob.glob('../20210724_exacloud_results/'+cohort+'*.tsv'))
    
    sample_counts = []
    average_scores = []
    resample_scores_by_step = []
    stdev_step = []
    for path in cohort_path_block:
        sample_size = int(path.split('.')[5])
        sample_counts.append(sample_size)

        file = pd.read_csv(path, sep = '\t')
        resample_scores = []
        test = file[file['Test'] == 1].copy()
        test.reset_index(inplace = True, drop = True)
        for i in test.Resampling.unique():

            scoreDF = test[test.Resampling == i]
            score = f1_score(scoreDF.iloc[:,4],
                     scoreDF.iloc[:,5],
                     average='weighted')
            resample_scores.append(score)
        stdev_step.append(statistics.stdev(resample_scores))
        resample_scores_by_step.append(resample_scores)    
        average_scores.append(mean(resample_scores))
    score_frame = pd.DataFrame([[cohort, sample_counts, average_scores,
                                 resample_scores_by_step, stdev_step]],
                           columns=['Cohort',
                                    'Sample_counts',
                                    'Average_scores',
                                    'Resample_by_step',
                                    'Stdev_step'])
    store_frame = pd.concat([store_frame, score_frame])
#     if cohort == 'BLCA':
#         break
store_frame.reset_index(inplace = True, drop = True)
end = time.time() - start
print(str(end))
print('all done')

144.905375957489
all done


In [106]:
type(store_frame.iloc[1,3]) # Check the raw resampling scores

list

In [99]:
store_frame.to_csv('Sample_count_accuracy_response_20210729.tsv', sep = '\t', index = False)

In [4]:
store_frame_test = pd.read_csv(
    'Sample_count_accuracy_response_20210729.tsv',
    sep = '\t')

In [5]:
type(eval(store_frame_test.iloc[1,3])) # Recheck the raw resampling scores

list

In [6]:
len(eval(store_frame_test.iloc[1,3])) # 25 lists of 100 scores for each sample size

25

In [8]:
len(eval(store_frame_test.iloc[1,3])[0]) # 100 resampling scores at the sample size of 10

100